In [2]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, dash_table
import dash_bootstrap_components as dbc

# Charger les données
df = pd.read_excel("C:/Users/LP/Desktop/analyst/Data Analysis/Excel Evaluation - Topic.xlsx", sheet_name='Part 1')
distributors = pd.read_excel("C:/Users/LP/Desktop/analyst/Data Analysis/Excel Evaluation - Topic.xlsx", sheet_name='Part 2')

# Manipulations de données
df['Total'] = df.iloc[:, 2:].sum(axis=1)
df['Average'] = df.iloc[:, 2:-1].mean(axis=1)
df['Min'] = df.iloc[:, 2:-2].min(axis=1)
df['Max'] = df.iloc[:, 2:-3].max(axis=1)
df['MoM'] = (df.iloc[:, -2] - df.iloc[:, -3]) / df.iloc[:, -3].replace(0, 1)

# Fusionner avec les données des distributeurs
df_merged = df.merge(distributors, on='MOVIE', how='left')

# Création du tableau croisé dynamique
pivot_table_distributors = df_merged.pivot_table(
    index='DISTRIBUTOR', 
    values='Total',
    aggfunc=['sum', 'mean']
)

# Aplatir le MultiIndex pour un accès plus facile dans Plotly
pivot_table_distributors.columns = ['_'.join(col).strip() for col in pivot_table_distributors.columns.values]

# Arrondir les valeurs à un chiffre après la virgule
pivot_table_distributors = pivot_table_distributors.round(1)

# Réinitialiser l'index pour transformer les index en colonne pour l'affichage dans Plotly
pivot_table_distributors.reset_index(inplace=True)

# Initialiser l'application Dash
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Style CSS pour les encadrements des graphiques et des statistiques
# Style CSS pour les encadrements des graphiques et des statistiques
card_style = {
    'border': '1px solid #007bff',
    'border-radius': '5px',
    'padding': '10px',
    'margin': '10px',
    'text-align': 'center'
}

# Layout de l'application
app.layout = dbc.Container([
    html.H1('Tableau de bord des données cinématographiques', style={'text-align': 'center'}),
    # Section pour les statistiques (moyenne, maximum, minimum)
    dbc.Row([
        dbc.Col(html.Div([
            html.H3("Moyenne"),
            html.H4(f"{df['Total'].mean():.1f}", style={'color': '#007bff'})
        ], style=card_style), md=4),
        dbc.Col(html.Div([
            html.H3("Maximum"),
            html.H4(f"{df['Total'].max():.1f}", style={'color': '#28a745'})
        ], style=card_style), md=4),
        dbc.Col(html.Div([
            html.H3("Minimum"),
            html.H4(f"{df['Total'].min():.1f}", style={'color': '#dc3545'})
        ], style=card_style), md=4)
    ]),
    # Section pour les graphiques
    dbc.Row([
        dbc.Col(html.Div([
            html.H3("Total par Film", style={'text-align': 'center'}),
            dcc.Graph(id='horizontal-bar-total-by-movie')
        ], style=card_style), md=6),
        dbc.Col(html.Div([
            html.H3("Moyenne par Genre", style={'text-align': 'center'}),
            dcc.Graph(id='vertical-bar-average-by-genre')
        ], style=card_style), md=6)
    ]),
    dbc.Row([
        dbc.Col(html.Div([
            html.H3("Moyenne par Distributeur", style={'text-align': 'center'}),
            dcc.Graph(id='pie-chart-average-by-distributor')
        ], style=card_style), md=6),
        dbc.Col(html.Div([
            html.H3("Changement MoM", style={'text-align': 'center'}),
            dcc.Graph(id='scatter-mom-change')
        ], style=card_style), md=6)
    ]),
    # ... [Le reste du layout et les callbacks restent inchangés]


# ... [Le reste du code reste inchangé]

  

    dbc.Row([
        dbc.Col(dash_table.DataTable(
            data=pivot_table_distributors.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in pivot_table_distributors.columns],
            style_table={'overflowX': 'auto'},
            style_cell={
                'height': 'auto',
                'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
                'whiteSpace': 'normal'
            },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ]
        ), md=12)
    ])
], fluid=True)

# Callbacks pour les graphiques
@app.callback(
    Output('horizontal-bar-total-by-movie', 'figure'),
    [Input('horizontal-bar-total-by-movie', 'id')]
)
def update_horizontal_bar(_):
    fig = px.bar(df, x='Total', y='MOVIE', orientation='h')
    return fig

@app.callback(
    Output('vertical-bar-average-by-genre', 'figure'),
    [Input('vertical-bar-average-by-genre', 'id')]
)
def update_vertical_bar(_):
    avg_by_genre = df.groupby('GENRE')['Average'].mean().reset_index()
    fig = px.bar(avg_by_genre, x='GENRE', y='Average')
    return fig

@app.callback(
    Output('pie-chart-average-by-distributor', 'figure'),
    [Input('pie-chart-average-by-distributor', 'id')]
)
def update_pie_chart(_):
    fig = px.pie(pivot_table_distributors, names='DISTRIBUTOR', values='sum_Total')
    return fig


@app.callback(
    Output('scatter-mom-change', 'figure'),
    [Input('scatter-mom-change', 'id')]
)
def update_scatter_mom_change(_):
    fig = px.scatter(df, x='Total', y='MoM', color='GENRE', size='Total', hover_name='MOVIE')
    return fig

# Exécuter l'application
if __name__ == '__main__':
    app.run_server(debug=True, port=5523)


In [7]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, dash_table
import dash_bootstrap_components as dbc

# Charger les données
df = pd.read_excel("C:/Users/LP/Desktop/analyst/Data Analysis/Excel Evaluation - Topic.xlsx", sheet_name='Part 1')
distributors = pd.read_excel("C:/Users/LP/Desktop/analyst/Data Analysis/Excel Evaluation - Topic.xlsx", sheet_name='Part 2')

# Manipulations de données
df['Total'] = df.iloc[:, 2:].sum(axis=1)
df['Average'] = df.iloc[:, 2:-1].mean(axis=1)
df['Min'] = df.iloc[:, 2:-2].min(axis=1)
df['Max'] = df.iloc[:, 2:-3].max(axis=1)
df['MoM'] = (df.iloc[:, -2] - df.iloc[:, -3]) / df.iloc[:, -3].replace(0, 1)

# Fusionner avec les données des distributeurs
df_merged = df.merge(distributors, on='MOVIE', how='left')

# Création du tableau croisé dynamique
pivot_table_distributors = df_merged.pivot_table(
    index='DISTRIBUTOR', 
    values='Total',
    aggfunc=['sum', 'mean']
)

# Aplatir le MultiIndex pour un accès plus facile dans Plotly
pivot_table_distributors.columns = ['_'.join(col).strip() for col in pivot_table_distributors.columns.values]

# Arrondir les valeurs à un chiffre après la virgule
pivot_table_distributors = pivot_table_distributors.round(1)

# Réinitialiser l'index pour transformer les index en colonne pour l'affichage dans Plotly
pivot_table_distributors.reset_index(inplace=True)

# Initialiser l'application Dash
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Style CSS pour les encadrements des graphiques et des statistiques
# Style CSS pour les encadrements des graphiques et des statistiques
# Style CSS pour les encadrements des graphiques
graph_card_style = {
    'border': '1px solid #007bff',
    'border-radius': '5px',
    'padding': '10px',
    'margin': '10px'
}

# Style CSS pour les encadrements des titres
# Style CSS pour les encadrements des titres
title_card_style = {
    'border': '1px solid #007bff',
    'border-radius': '5px',
    'padding': '5px',
    'margin-bottom': '5px',
    'text-align': 'center',
    'background-color': '#f8f9fa',  # Couleur de fond pour les titres
    'font-weight': 'bold',  # Ajout pour le gras
    'text-transform': 'uppercase'  # Ajout pour les majuscules
}

# Layout de l'application
app.layout = dbc.Container([
    html.H1('Tableau de bord des données cinématographiques', style={'text-align': 'center'}),
    # Section pour les statistiques (moyenne, maximum, minimum)
    dbc.Row([
        
    ]),

    
    # Section pour les graphiques
    dbc.Row([
        dbc.Col(html.Div([
            html.Div("Total par film", style=title_card_style),
            dcc.Graph(id='horizontal-bar-total-by-movie')
        ], style=graph_card_style), md=6),
        dbc.Col(html.Div([
            html.Div("Moyenne par genre", style=title_card_style),
            dcc.Graph(id='vertical-bar-average-by-genre')
        ], style=graph_card_style), md=6)
    ]),
    dbc.Row([
        dbc.Col(html.Div([
            html.Div("Moyenne par distributeur", style=title_card_style),
            dcc.Graph(id='pie-chart-average-by-distributor')
        ], style=graph_card_style), md=6),
        dbc.Col(html.Div([
            html.Div("Changement mom", style=title_card_style),
            dcc.Graph(id='scatter-mom-change')
        ], style=graph_card_style), md=6)
    ]),
  
######


    dbc.Row([
        dbc.Col(dash_table.DataTable(
            data=pivot_table_distributors.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in pivot_table_distributors.columns],
            style_table={'overflowX': 'auto'},
            style_cell={
                'height': 'auto',
                'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
                'whiteSpace': 'normal'
            },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ]
        ), md=12)
    ])
    
], fluid=True)

# Callbacks pour les graphiques
@app.callback(
    Output('horizontal-bar-total-by-movie', 'figure'),
    [Input('horizontal-bar-total-by-movie', 'id')]
)
def update_horizontal_bar(_):
    fig = px.bar(df, x='Total', y='MOVIE', orientation='h')
    return fig

@app.callback(
    Output('vertical-bar-average-by-genre', 'figure'),
    [Input('vertical-bar-average-by-genre', 'id')]
)
def update_vertical_bar(_):
    avg_by_genre = df.groupby('GENRE')['Average'].mean().reset_index()
    fig = px.bar(avg_by_genre, x='GENRE', y='Average')
    return fig

@app.callback(
    Output('pie-chart-average-by-distributor', 'figure'),
    [Input('pie-chart-average-by-distributor', 'id')]
)
def update_pie_chart(_):
    fig = px.pie(pivot_table_distributors, names='DISTRIBUTOR', values='sum_Total')
    return fig


@app.callback(
    Output('scatter-mom-change', 'figure'),
    [Input('scatter-mom-change', 'id')]
)
def update_scatter_mom_change(_):
    fig = px.scatter(df, x='Total', y='MoM', color='GENRE', size='Total', hover_name='MOVIE')
    return fig

# Exécuter l'application
if __name__ == '__main__':
    app.run_server(debug=True, port=5523)
